In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv")


In [ ]:
df.head()

In [ ]:
df.shape # shape of data how many row`s and columns in you data

In [ ]:
df.isnull().sum() # How many missing values in your dataset

In [ ]:
df.dtypes # what is your data types in your data set

In [ ]:
df.sample()['clean_comment'].values

In [ ]:
df[df['clean_comment'].isna()]

In [ ]:
df[df['clean_comment'].isna()]['category'].value_counts() # how many rowa missing which is category only 0 = neutral comment category comment missing values founds

In [ ]:
df['category'].value_counts()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
df['clean_comment']=df['clean_comment'].str.lower().str.strip()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(subset=['clean_comment', 'category'], inplace=True, keep='first')

In [ ]:
df.duplicated().sum()

In [ ]:
duplicate_rows=df[df.duplicated()]

In [ ]:
duplicate_rows['category'].value_counts()

In [ ]:
df[df['clean_comment'].apply(lambda x: x.endswith(' ') or x.startswith(' '))].sum()

In [ ]:

# Remove trailing and leading whitespaces from the 'clean_comment' column
df['clean_comment'] = df['clean_comment'].str.strip()

# Verify the transformation by checking for any remaining trailing whitespaces
df['clean_comment'].apply(lambda x: x.endswith(' ') or x.startswith(' ')).sum()
     

In [ ]:

# Identify comments containing URLs
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

comments_with_url=df['clean_comment'].str.contains(url_pattern, regex=True)

In [ ]:
df[comments_with_url]

In [ ]:
comment_with_newline=df[df['clean_comment'].str.contains('\n')]

In [ ]:
comment_with_newline

In [ ]:
df = df[df['clean_comment'].str.len() > 0]  # Empty comments drop karo

In [ ]:
df['clean_comment']=df['clean_comment'].str.replace('\n', ' ', regex=True)

In [ ]:
df[df['clean_comment'].str.contains('\n')]

In [ ]:
# EDA

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.countplot(data=df , x='category')

In [ ]:
df['category'].value_counts(normalize=True).mul(100).round(2)

In [ ]:
df['word_count']=df['clean_comment'].apply(lambda x: len(x.split()))

In [ ]:
df

In [ ]:
df['word_count'].describe()

In [ ]:
sns.displot(df['word_count'], kde=True)

In [ ]:
# Empty comments check
print("Empty comments count:", len(df[df['clean_comment'].str.len() == 0]))

In [ ]:
plt.figure(figsize=(6,6))

sns.kdeplot(df[df['category']==1]['word_count'], label="Positive", fill=True)
sns.kdeplot(df[df['category']==0]['word_count'], label="Positive", fill=True)
sns.kdeplot(df[df['category']==-1]['word_count'], label="Positive", fill=True)

plt.legend()
plt.show()

In [ ]:
sns.boxplot(df)

In [ ]:
plt.figure(figsize=(5,3))
sns.boxplot(data=df, x='category', y='word_count', fill=True)
plt.show()

In [ ]:
sns.scatterplot(data=df, x='word_count', y='category')

In [ ]:
# median word counts among sentiments

sns.barplot(df,x='category',y='word_count',estimator='median')

In [ ]:

! pip install nltk

In [ ]:
import nltk
from nltk.corpus import stopwords

# Stopwords download karo (pehli baar ke liye)
nltk.download('stopwords')

# English ke stopwords
stop_words = list(stopwords.words('english'))

# Apna text
text = "This is a sample sentence showing off the stopwords."
words = text.split()

# Stopwords hatao
filtered = [word for word in words if word.lower() not in stop_words]

print(filtered)
# Output: ['sample', 'sentence', 'showing', 'stopwords', '.']


In [ ]:
print(stop_words)

In [ ]:
df['stop_words']=df['clean_comment'].apply(lambda x: len([word for word in x.split() if word in stop_words]))

In [ ]:
df.sample(13)

In [ ]:

# Create the figure and axes
plt.figure(figsize=(10, 6))

# Plot KDE for category 1
sns.kdeplot(df[df['category'] == 1]['stop_words'], label='Positive', fill=True)
sns.kdeplot(df[df['category'] == 0]['stop_words'], label='neutral', fill=True)
sns.kdeplot(df[df['category'] == -1]['stop_words'], label='negative', fill=True)


In [ ]:

# median word counts among sentiments

sns.barplot(df,x='category',y='stop_words',estimator='mean')

In [ ]:
df[df['category']==0].describe()

In [ ]:
print(df[df['category']==1].describe())
print(df[df['category']==-1].describe())

In [ ]:
from collections import Counter

all_stop_words=[word for comment in df['clean_comment'] for word in comment.split() if word in stop_words]


In [ ]:
most_common_stop_word=Counter(all_stop_words).most_common(25)

top_25_words=pd.DataFrame(most_common_stop_word, columns=['stop_word','count'])

In [ ]:
plt.figure(figsize=(10,10))

sns.barplot(data=top_25_words, x='count', y='stop_word', palette='viridis')
plt.title('Top 25 Most Common Stop Words')
plt.xlabel('Count')
plt.ylabel('Stop Word')
plt.show()

In [ ]:

df['num_chars'] = df['clean_comment'].apply(len)

df.head()
     

In [ ]:
all_text= " ".join(df['clean_comment'])

In [ ]:
char_freq=Counter(all_text)

In [ ]:
st="Muhammad Inam is the AI engineer 2026"

sty=Counter(st)

sty.items()

In [ ]:
char_frequency=pd.DataFrame(char_freq.items(), columns=["charater", "frequency"])

In [ ]:
char_frequency['charater'].values

In [ ]:

char_frequency.tail(50)
     

In [ ]:
df["num_punctuation_column"]=df['clean_comment'].apply(lambda x: sum([1 for char in x if char in '.,!?;:"\'()[]{}-']))

In [ ]:
df

In [ ]:
! pip install sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_ngram(corpus, n=None):
    vectorizer=CountVectorizer(ngram_range=(2,2), stop_words='english').fit(corpus)
    bag_of_words = vectorizer.transform(corpus)
    sum_of_word=bag_of_words.sum(axis=0)